In [1]:
import ppuu.lightning_modules
import ppuu.train_policy
import ppuu.costs.policy_costs_km
import imp
imp.reload(ppuu.lightning_modules)
imp.reload(ppuu.train_policy)
imp.reload(ppuu.costs.policy_costs_km)
Module = ppuu.lightning_modules.MPURDreamingModule

In [2]:
import random

config = Module.Config()
config.model_config.model_type = 'dreaming'
config.training_config.set_dataset('50')
config.training_config.epoch_size = 10
config.training_config.n_epochs = 1
config.training_config.seed = random.randint(0, 1000)
config.training_config.batch_size = 1
config.training_config.validation_size = 10
config.training_config.output_dir = '/home/us441/vlad/garbage'
config.training_config.freeze_encoder = True
config.training_config.mixout_p = 0.5
config.training_config.validation_eval = False
config.model_config.checkpoint = '/misc/vlgscratch4/LecunGroup/nvidia-collab/vlad/results/refactored_test/continuous/seed=1/checkpoints/epoch=60.ckpt'
config.cost_config.uncertainty_n_batches = 10 
model = ppuu.train_policy.main(config)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


loading from the checkpoint



  | Name          | Type                      | Params
------------------------------------------------------------
0 | forward_model | ForwardModel              | 15 M  
1 | policy_model  | MixoutDeterministicPolicy | 6 M   
INFO:lightning:
  | Name          | Type                      | Params
------------------------------------------------------------
0 | forward_model | ForwardModel              | 15 M  
1 | policy_model  | MixoutDeterministicPolicy | 6 M   


pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/us441/vlad/miniconda3/envs/forked/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 44 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/us441/vlad/miniconda3/envs/forked/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 44 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


[estimating normal uncertainty ranges: 100.0%]


In [3]:
model.policy_model.fc[0].weight

Parameter containing:
tensor([[ 0.0355, -0.0411,  0.0202,  ..., -0.0190,  0.0136, -0.0148],
        [ 0.0080,  0.0280, -0.0118,  ..., -0.0260, -0.0320,  0.0626],
        [ 0.0092, -0.0309, -0.0134,  ...,  0.0490, -0.0655,  0.0052],
        ...,
        [ 0.0259, -0.0124,  0.0367,  ...,  0.0315, -0.0242, -0.0579],
        [ 0.0392, -0.0513,  0.0698,  ..., -0.0232,  0.0048, -0.0288],
        [-0.0171,  0.0188, -0.0023,  ...,  0.0580, -0.0183, -0.0529]],
       device='cuda:0', requires_grad=True)

In [7]:
model.policy_model.original_model.encoder.f_encoder[0].weight

Parameter containing:
tensor([[[[ 5.5778e-03, -4.2797e-03,  5.6383e-03,  5.6718e-03],
          [-3.3398e-02,  2.5219e-02,  2.4000e-03,  5.2392e-03],
          [-1.3728e-03,  4.3157e-03,  1.3141e-02, -9.0844e-03],
          [ 6.1811e-03, -3.0375e-03,  7.0779e-04, -1.5422e-02]],

         [[-9.3778e-03,  4.4887e-03,  3.9189e-02,  1.2141e-02],
          [-2.8638e-02, -1.7446e-02,  2.8173e-03, -1.4944e-02],
          [-2.7832e-02, -2.1329e-03,  1.7539e-02,  6.9513e-03],
          [-5.5775e-02,  1.3499e-03, -7.3869e-03,  5.4745e-03]],

         [[ 2.7825e-02, -2.4729e-02, -3.5918e-02, -1.9341e-02],
          [ 3.3095e-02, -5.0762e-03,  6.5722e-05, -2.0674e-02],
          [ 3.5265e-02, -1.3992e-02,  7.2216e-03, -9.4212e-03],
          [-7.9648e-03,  1.0198e-02, -3.1026e-02, -3.8141e-03]],

         ...,

         [[ 9.1227e-02,  1.0772e-01,  8.9870e-02, -2.0448e-02],
          [ 6.0867e-02,  1.1615e-01,  7.0178e-02, -1.8370e-02],
          [ 5.7364e-02,  8.6911e-02,  6.4821e-02, -4.0500e-02

In [ ]:
import imp
import ppuu
from ppuu.modeling.mixout import MixLinear

s = []
for layer in model.policy_model.fc:
    if isinstance(layer, torch.nn.Linear):
        s.append(MixLinear(layer.in_features, layer.out_features, bias=True, target=layer.weight, p=0.5))
    else:
        s.append(layer)
print(s)

In [20]:
print(model.policy_model.fc)

Sequential(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=2, bias=True)
)
